## 4.2. 模型参数的访问、初始化和共享
在“线性回归的简洁实现”一节中，我们通过`init`模块来初始化模型的全部参数。我们也介绍了访问模型参数的简单方法。本节将深入讲解如何访问和初始化模型参数，以及如何在多个层之间共享同一份模型参数。

我们先定义一个与上一节中相同的含单隐藏层的多层感知机。我们依然使用默认方式初始化它的参数，并做一次前向计算。

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
print(tf.__version__)

2.1.0


In [2]:
net = keras.Sequential()
net.add(keras.layers.Flatten())
net.add(keras.layers.Dense(256, activation='relu'))
net.add(keras.layers.Dense(10))

X = tf.random.uniform(shape=(2, 20))
Y = net(X)  # 前向计算
Y

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.11525244, -0.13519563,  0.20563626,  0.1947599 , -0.06194043,
        -0.22070742,  0.18779312,  0.1698063 ,  0.12842052, -0.18753651],
       [ 0.3472902 , -0.18563515,  0.3329596 ,  0.12333538, -0.23827925,
        -0.10464031,  0.16165382,  0.4064858 ,  0.06976099, -0.11919112]],
      dtype=float32)>

### 4.2.1. 访问模型参数
对于使用`Sequential`类构造的神经网络，我们可以通过`weights`属性来访问网络任一层的权重。回忆一下上一节中提到的`Sequential`类与`tf.keras.Model`类的继承关系。对于`Sequential`实例中含模型参数的层，我们可以通过`tf.keras.Model`类的`weights`属性来访问该层包含的所有参数。下面，访问多层感知机`net`中隐藏层的所有参数。索引`0`表示隐藏层为`Sequential`实例最先添加的层。

In [3]:
net.weights[0],type(net.weights[0])

(<tf.Variable 'sequential/dense/kernel:0' shape=(20, 256) dtype=float32, numpy=
 array([[-0.07069016,  0.11090916, -0.13917884, ...,  0.05293907,
         -0.09434916,  0.08805928],
        [-0.1346477 ,  0.10465711, -0.02740826, ..., -0.04868089,
         -0.09580112,  0.11522248],
        [-0.03858206,  0.06133232,  0.09340487, ...,  0.12659445,
         -0.07707801, -0.1015819 ],
        ...,
        [ 0.10917932,  0.1103344 ,  0.03767487, ...,  0.10450956,
          0.08945026,  0.02449542],
        [ 0.0456754 ,  0.0581727 ,  0.0269246 , ...,  0.09464845,
         -0.11656308, -0.09958023],
        [ 0.04684894, -0.00157274, -0.03242975, ...,  0.1232672 ,
         -0.1369152 ,  0.0503927 ]], dtype=float32)>,
 tensorflow.python.ops.resource_variable_ops.ResourceVariable)

In [4]:
net.weights

[<tf.Variable 'sequential/dense/kernel:0' shape=(20, 256) dtype=float32, numpy=
 array([[-0.07069016,  0.11090916, -0.13917884, ...,  0.05293907,
         -0.09434916,  0.08805928],
        [-0.1346477 ,  0.10465711, -0.02740826, ..., -0.04868089,
         -0.09580112,  0.11522248],
        [-0.03858206,  0.06133232,  0.09340487, ...,  0.12659445,
         -0.07707801, -0.1015819 ],
        ...,
        [ 0.10917932,  0.1103344 ,  0.03767487, ...,  0.10450956,
          0.08945026,  0.02449542],
        [ 0.0456754 ,  0.0581727 ,  0.0269246 , ...,  0.09464845,
         -0.11656308, -0.09958023],
        [ 0.04684894, -0.00157274, -0.03242975, ...,  0.1232672 ,
         -0.1369152 ,  0.0503927 ]], dtype=float32)>,
 <tf.Variable 'sequential/dense/bias:0' shape=(256,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [5]:
net.weights[0].numpy(),net.weights[0].numpy().shape

(array([[-0.07069016,  0.11090916, -0.13917884, ...,  0.05293907,
         -0.09434916,  0.08805928],
        [-0.1346477 ,  0.10465711, -0.02740826, ..., -0.04868089,
         -0.09580112,  0.11522248],
        [-0.03858206,  0.06133232,  0.09340487, ...,  0.12659445,
         -0.07707801, -0.1015819 ],
        ...,
        [ 0.10917932,  0.1103344 ,  0.03767487, ...,  0.10450956,
          0.08945026,  0.02449542],
        [ 0.0456754 ,  0.0581727 ,  0.0269246 , ...,  0.09464845,
         -0.11656308, -0.09958023],
        [ 0.04684894, -0.00157274, -0.03242975, ...,  0.1232672 ,
         -0.1369152 ,  0.0503927 ]], dtype=float32),
 (20, 256))

### 4.2.2. 初始化模型参数
我们在\[“数值稳定性和模型初始化”\]一节中描述了模型的默认初始化方法：权重参数元素为\[-0.07, 0.07\]之间均匀分布的随机数，偏差参数则全为0。但我们经常需要使用其他方法来初始化权重。在下面的例子中，我们将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。(暂时不知道`keras`有重新初始化参数的方式)

In [6]:
net = keras.Sequential()
net.add(keras.layers.Flatten())
net.add(keras.layers.Dense(256, activation='relu',kernel_initializer=keras.initializers.RandomNormal(mean=0,stddev=0.01)
                           ,bias_initializer="zeros"))
net.add(keras.layers.Dense(10,kernel_initializer=keras.initializers.RandomNormal(mean=0,stddev=0.01)))
net(X)
net.get_weights()

[array([[ 0.00057808,  0.01212484,  0.02041551, ...,  0.00143753,
         -0.01051395,  0.00880218],
        [-0.00270523,  0.00919028, -0.00500893, ..., -0.00744017,
         -0.00373847,  0.00638111],
        [-0.00155448,  0.00470958,  0.01261406, ..., -0.00096021,
         -0.01647384,  0.0059849 ],
        ...,
        [-0.00133069,  0.00715045,  0.0028836 , ...,  0.01253994,
          0.00179943, -0.0065921 ],
        [-0.01062844,  0.01658173, -0.0018872 , ..., -0.00448022,
         -0.01040471,  0.00342269],
        [-0.01169182,  0.01723223, -0.00309148, ..., -0.011168  ,
         -0.00874047,  0.00666463]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

下面使用常数来初始化权重参数。

In [7]:
net = keras.Sequential()
net.add(keras.layers.Flatten())
net.add(keras.layers.Dense(256, activation='relu',kernel_initializer=keras.initializers.Constant(1)
                           ,bias_initializer="zeros"))
net.add(keras.layers.Dense(10,kernel_initializer=keras.initializers.RandomNormal(mean=0,stddev=0.01)))
net(X)
net.get_weights()

[array([[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

Glorot正态分布初始化方法，也称作`Xavier`正态分布初始化，参数由`0`均值，标准差为`sqrt(2 / (fan_in + fan_out))`的正态分布产生，其中`fan_in`和`fan_out`是权重张量的扇入扇出（即输入和输出单元数目）。下例中我们对隐藏层的权重使用`Xavier`随机初始化方法。

In [8]:
net = keras.Sequential()
net.add(keras.layers.Flatten())
net.add(keras.layers.Dense(256, activation='relu',kernel_initializer=keras.initializers.GlorotNormal()
                           ,bias_initializer="zeros"))
net.add(keras.layers.Dense(10,kernel_initializer=keras.initializers.RandomNormal(mean=0,stddev=0.01)))
net(X)
net.get_weights()

[array([[ 0.0695044 , -0.07252984,  0.16599984, ..., -0.01858299,
         -0.03362315, -0.13336617],
        [ 0.07669634,  0.10013129,  0.02140373, ...,  0.01952465,
         -0.11397454,  0.02004574],
        [-0.10765048, -0.11513884, -0.03648057, ...,  0.00914257,
          0.1434916 ,  0.02981399],
        ...,
        [ 0.0114043 ,  0.01820433, -0.01351871, ..., -0.12122075,
         -0.12489499,  0.06077783],
        [ 0.0642491 ,  0.02380809,  0.12315745, ...,  0.10360536,
          0.11231796,  0.06345495],
        [-0.1374723 ,  0.06293916, -0.09724572, ..., -0.1000086 ,
          0.0846191 ,  0.15534091]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

### 4.2.3. 自定义初始化方法
可以使用`tf.keras.initializers`类中的方法实现自定义初始化。如果需要传递自定义的初始化器，则该初始化器必须是`callable`的，并且接收`shape`（将被初始化的张量`shape`）和`dtype`（数据类型）两个参数，并返回符合`shape`和`dtype`的张量。

In [9]:
from tensorflow.keras import backend as K

def my_init(shape, dtype=None):
    return K.random_normal(shape, dtype=dtype)

net = keras.Sequential()
net.add(keras.layers.Flatten())
net.add(keras.layers.Dense(256, activation='relu',kernel_initializer=my_init
                           ,bias_initializer="zeros"))
net.add(keras.layers.Dense(10,kernel_initializer=keras.initializers.RandomNormal(mean=0,stddev=0.01)))
net(X)
net.get_weights()

[array([[-0.0890545 ,  0.4419847 , -0.9641655 , ..., -0.12421888,
          0.3267785 , -0.6040333 ],
        [-1.0615999 , -1.0682908 ,  0.21529667, ..., -0.03203521,
          1.9521668 , -0.7281848 ],
        [-0.34160733, -0.5677692 , -1.2416878 , ...,  1.0285121 ,
          0.43271908,  0.32829037],
        ...,
        [-1.7969391 ,  0.3126901 ,  0.21185884, ..., -0.24406238,
          0.6393728 , -0.0757916 ],
        [ 1.8315326 , -1.1442302 , -0.14809616, ...,  1.1795167 ,
          0.43464005,  0.06140326],
        [ 0.21035595,  0.09683031,  2.2004168 , ...,  1.7135043 ,
         -0.45960483,  0.53353256]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.